In [112]:
#list of subreddits
#Abandond, art, Be_amazed, crafts, creepy, design, earth_p, eye_bleach, fishing, food_p, gardening, hiking, interesting,
#interesting_af, mildly_infuriating, mildly_interesting,oddly_terrifying, pics, rare_puppers, skin_care, space, tattoo, trashy,
#weird

In [113]:
subreddit = "crafts"

In [114]:
import pandas as pd
from nltk.corpus import words
import numpy as np
import nltk.corpus
import spacy
import re
import string
from spacy_langdetect import LanguageDetector
#rom autocorrect import Speller
#spell = Speller()
from profanity_filter import ProfanityFilter
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True) #2
#ltk.download('words')
from langdetect import detect, detector_factory, detect_langs

In [115]:
DATA = f'C:/Users/Charlie/Desktop/Database/BERT-Sentiment-Analysis-Reddit-API/DataCleaning/filteredData/{subreddit}.csv'

In [116]:
data = pd.read_csv(DATA,sep=",", encoding='cp1252',on_bad_lines='skip')
#cp1252
#utf-8
data_first = pd.read_csv(DATA,sep=",", encoding='cp1252',on_bad_lines='skip')[['Column1', 'Column2', 'Column3']]
data_first
#data_first.to_csv(f'C:/Users/Charlie/Desktop/Database/BERT-Sentiment-Analysis-Reddit-API/DataCleaning/filteredData/{subreddit}_image_links.csv', index=False, encoding='utf8')

In [117]:
data_subset = data.drop(['Column1', 'Column2', 'Column3'], axis=1)
data_compare = data.drop(['Column1', 'Column2', 'Column3'], axis=1)
data_subset

,Column4,Column5,Column6,Column7,Column8,Column9,Column10,Column11,Column12,Column13,...,Column44,Column45,Column46,Column47,Column48,Column49,Column50,Column51,Column52,Column53
0,2,3,4,5,6,7,8,9,10,1,...,42,43,4,45,46,47,48,49,50,51
1,"This is amazing, does she have a website?",Incredible how he manages to stay still for so...,I love how pleased he looks with himself. Be...,Your sister is a wizard,Ph'nglui mglw'nafh Cthulhu R'lyeh wgah'nagl fh...,removed,Those eyes bring back fond memories!,Shut up and take my money!,This was handmade by my oldest daughter. She d...,I need this for Hanukah. This would make an aw...,...,It looks so happy to be useful by holding the ...,I've always prefered this one.,I think your sister might be into tentacle ****.,"I can't believe she made this by hand, without...",pottery CLAS? is she the teacher? calling b...,Absolutely phenomenal. Would love to see her p...,But what about your sister wives?,Hey I saw your sister A post this on Facebook ...,Yikes! My comment came out wacky! My daughter ...,Contact her at Nnyl_ahsatan1
2,Since I saw someone else posting this the othe...,You should post this at,These are amazing! So much attention to detail,"Wow, these are incredible! What got you starte...",Oh wow- I might have commented on that complim...,Stoner food? Edit came back to say I love the...,These are awesome. Also I love that duff and c...,"Great work Jen, you are a true artist!","These are great! For the letters, do you use p...",You are so skilled! Do you hide the rocks in p...,...,Fluff is my fave,I love this so much,Omg! This is really good! Now I want to start ...,I would break all my teeth. Just one after ano...,It truly is super sharp!,This is adorably brilliant!,"This would make a great poster,",that bologna rock lookin like a SNACK,Wow! These are fantastic! And they way this ph...,How cool!
3,This is so freaking cool! I'm tempted to do th...,You are a capital G Genius! I don't have li...,Any kind of markers? Supervisor of a special ...,This is such a good idea for the little ones. ...,The shadows are super pretty but I would be mi...,I love this! And I just looked up “black paint...,I swear my first thought was “I bet that marke...,Just wow! Love the creativity!,I saw this on my neighbors window yesterday on...,Genius! And it looks so pretty!,...,This makes more sense! Someone on my facebook ...,This is awesome!,Such a cool idea.,It’s so lovely!,"It doesn't have to be pretty, to be beautiful....",That is quite beautiful,Brilliant!,So cool!,Excuse me while I transform my house.,Absolutely loving the idea! And it's so beauti...
4,You did an amazing job!,That's so amazing! Everything looks so acurate...,It's an heirloom now!,I was wondering where you got the pattern from...,WHAT THE HECK! this is amazing! i’m so impress...,Amazing dedication to finishing it! Your hands...,Oh I think you missed a thread! LOL.kiding! Th...,At first I thought “ that’s one **** of a nose...,“ITS TERIBLE” .said no one ever,"Looks amazing, you did a great job. How about ...",...,WOWOWOWOWOW!,This is literally amazing! One of the most bea...,This is insanely beautiful!,Wow!,That's absolutely incredible! What an outstand...,That monkey fina get got when he gets his *** ...,That is AMAZING!,WOW amazing. Look at all those colours. what a...,"That is beautiful, I would buy",This is amazing! You are incredibly talented! ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650,It's stuning,Lovely! Please share on too!,"Your wife is literally SO talented, I loved he...",WOW! So amazing! Does she sell her work anywhe...,This is really beautiful. How much does someth...,Beautiful!,Impressive,She did an amazing job!,Amazing! I know how much time & patience that ...,That is beautiful - thanks for sharing!,...,Thank you so much for the love guys. a special...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
651,It’s so pretty! It looks so soft too! What did...,Did you make this after watching Chris

In [118]:
#Removes emoticons
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

#removes characters that appear more than twice ex: booo -> boo
keep_list = ['s','e','t','f','l','m','o','p','d','n','g','r','b']
def remove_dupes(s):
    ans = ""
    seen = ''
    i = 0
    while i != (len(s)-1):
        if s[i] in keep_list and s[i] != seen:
            if s[i] == s[i+1]:
                seen = s[i]
                ans+= s[i]
                ans+= s[i+1]
        if s[i] != seen and s[i+1] != s[i]:
            seen = s[i]
            ans += s[i]
            i += 1
        else:
            i += 1
    ans += s[len(s)-1]
    if ans[-1] == ans[-2] and ans[-1] == ans[-2]:
        ans = ans[0:-1]
    return ans

#language detection:

def language(text):
    count = 0
    threshold = 3
    span = 4
    split = splitter(text,span)
    for word in split:
        print(word)
        doc = nlp(word)
        detect_language = doc._.language
        lang = detect_language['language']
        if lang != 'en':
            print(lang)
            count += 1
    if count >= threshold:
        return False
    else:
        return True
    
def splitter(text, span):
    s = span
    words = re.sub(r'[^\w\s]', '', text).split(' ')
    t = [" ".join(words[i:i+s]) for i in range(0, len(words), s)]
    return t
    
def language2(text):
    count = 0
    threshold = 2
    span = 3
    split = splitter(text,span)
    detector_factory.seed = 42
    for words in split:
        print(words)
        lang = detect(words)
        print(detect_langs(words), detect(words))
        if lang != 'en':
            print(lang)
            count += 1
    if count >= threshold:
        return False
    else:
        return True

#removes gibberish (Doesn't work, removes lots of real words as well)
'''
words = set(nltk.corpus.words.words())
def clean(sent):
    return " ".join(w for w in nltk.wordpunct_tokenize(sent) \
     if w.lower() in words or not w.isalpha())

text = 'building fence good mighty banana wiugSFNDnrgkdf help'
print(clean(text))
'''

'\nwords = set(nltk.corpus.words.words())\ndef clean(sent):\n    return " ".join(w for w in nltk.wordpunct_tokenize(sent)      if w.lower() in words or not w.isalpha())\n\ntext = \'building fence good mighty banana wiugSFNDnrgkdf help\'\nprint(clean(text))\n'

In [119]:
text = "bad."
language2(text)

bad
[so:0.9999971949631348] so
so


True

In [120]:
#Subsets dataset and define filters
data_subset = data.drop(['Column1', 'Column2', 'Column3'], axis=1)
delete_list = ['[deleted]','[removed]','removed','deleted','https', 'Repost','repost','!gif','&#x20B','[OC]','*',
              'instagram','facebook','twitter','google']
sub_list = [r'r/\S+',r'\([^)]*\)',r'http\S+',r'[()]',r'[:]',r'[\([{})\]]']
pf = ProfanityFilter()

for i in range(len(data_subset)):
#for i in range(1,4):
    for j in range(len(data_subset.iloc[i])):
        text = data_subset.iloc[i][j]
        try:
            text = remove_emojis(text)
        except:
            continue
            
        try:
            text = remove_dupes(text)
        except:
            print('dupe removal failed')
            continue
        text = text.replace("\r", " ").replace("\n", " ").replace("\t", " ").replace("_x000D_", " ").strip()
        #text = spell(text)
        #text = clean(text)
        try:
            text = re.sub(r'\([^)]*\)', '', text)
            for sub in sub_list:
                text = re.sub(sub,'',text)
        except:
            continue
        text = pf.censor(text)
        if any(word in data_subset.iloc[i][j].lower() for word in delete_list):
            text = 0
            
        #if text!=0:
            #if language(text) == False:
                #text = 0
        data_subset['Column' + str(j+4)][i] = text
    print(i,"out of",len(data_subset))

dupe removal failed
dupe removal failed
dupe removal failed
dupe removal failed
dupe removal failed
dupe removal failed
dupe removal failed
dupe removal failed
dupe removal failed
dupe removal failed
dupe removal failed
dupe removal failed
0 out of 655
1 out of 655
2 out of 655
dupe removal failed
3 out of 655
dupe removal failed
4 out of 655
5 out of 655
dupe removal failed
6 out of 655
7 out of 655
8 out of 655
9 out of 655
10 out of 655
11 out of 655
12 out of 655
13 out of 655
14 out of 655
15 out of 655
16 out of 655
17 out of 655
18 out of 655
19 out of 655
20 out of 655
dupe removal failed
21 out of 655
22 out of 655
23 out of 655
24 out of 655
dupe removal failed
25 out of 655
26 out of 655
27 out of 655
dupe removal failed
28 out of 655
29 out of 655
30 out of 655
31 out of 655
32 out of 655
dupe removal failed
33 out of 655
34 out of 655
35 out of 655
dupe removal failed
36 out of 655
37 out of 655
38 out of 655
39 out of 655
40 out of 655
41 out of 655
dupe removal failed
du

420 out of 655
421 out of 655
422 out of 655
423 out of 655
424 out of 655
425 out of 655
426 out of 655
dupe removal failed
427 out of 655
428 out of 655
429 out of 655
430 out of 655
431 out of 655
dupe removal failed
432 out of 655
433 out of 655
dupe removal failed
434 out of 655
435 out of 655
436 out of 655
437 out of 655
dupe removal failed
438 out of 655
439 out of 655
dupe removal failed
440 out of 655
441 out of 655
dupe removal failed
442 out of 655
443 out of 655
dupe removal failed
444 out of 655
dupe removal failed
445 out of 655
446 out of 655
447 out of 655
448 out of 655
449 out of 655
dupe removal failed
450 out of 655
451 out of 655
452 out of 655
453 out of 655
454 out of 655
455 out of 655
dupe removal failed
456 out of 655
457 out of 655
dupe removal failed
458 out of 655
459 out of 655
460 out of 655
461 out of 655
462 out of 655
dupe removal failed
463 out of 655
464 out of 655
465 out of 655
466 out of 655
467 out of 655
468 out of 655
469 out of 655
470 out of

In [121]:
#Duplicate text removal testing
#text = 'r9h32w9EAJFIDOSNGFMAJIWEMFODimns hello fifwoeldjgm'

#text = 'bluuuuuue abysssss, trrrrolllllllllllllllllll, heelllllpppp meeeee, yyellowww,,,, !!!!,#@$$$'
#remove_dupes(text)

#len(text)
#language(text)

In [122]:
#compares raw vs filtered data

index = i
for i in range(len(data_subset.iloc[index])):
    print ('old:',data_compare.iloc[index][i])
    print ('new:',data_subset.iloc[index][i])
    print(" ")
    


old: this messes with my brain in the very best way
new: this messes with my brain in the very best way
 
old: I couldn't even replicate a single blade of grass. Super work! Love the purple-blue hues.
new: I couldn't even replicate a single blade of grass. Super work! Love the purple-blue hues.
 
old: How did you get into this medium, and how many years of practice are we looking at?     I absolutely love this. This is true art and I adore that it exists.
new: How did you get into this medium, and how many years of practice are we looking at? I absolutely love this. This is true art and I adore that it exists.
 
old: This makes my soul happy
new: This makes my soul happy
 
old: Wow. Wow, wow, wow. WOW.     Asolutely beautiful!
new: Wow. Wow, wow, wow. WOW. Asolutely beautiful!
 
old: Looking at this is like tripping on acid
new: Looking at this is like tripping on acid
 
old: Holy cow this is amazing!
new: Holy cow this is amazing!
 
old: It's like a Monet that pops out at you. Very co

In [123]:
result = pd.concat([data_first, data_subset], axis=1, join='inner')
result = result.iloc[1:, :]
for i in range(1,len(result.columns)+1):
    if i == 1:
        result.rename(columns={f'Column{i}': 'index'}, inplace=True)
    if i == 2:
        result.rename(columns={f'Column{i}': 'title'}, inplace=True)
    if i == 3:
        result.rename(columns={f'Column{i}': 'image_link'}, inplace=True)
    else:
        result.rename(columns={f'Column{i}': f'comment{i-3}'}, inplace=True)

result

,index,title,image_link,comment1,comment2,comment3,comment4,comment5,comment6,comment7,...,comment41,comment42,comment43,comment44,comment45,comment46,comment47,comment48,comment49,comment50
1,0.0,This octopus candle holder that my sister hand...,https://i.redd.it/mg4ge4iqymlz.jpg,"This is amazing, does she have a website?",Incredible how he manages to stay still for so...,I love how pleased he looks with himself. Beau...,Your sister is a wizard,Ph'nglui mglw'nafh Cthulhu R'lyeh wgah'nagl fh...,0,Those eyes bring back fond memories!,...,0,I've always prefered this one.,0,"I can't believe she made this by hand, without...",pottery CLAS? is she the teacher? calling bs op,Absolutely phenomenal. Would love to see her p...,But what about your sister wives?,0,0,Contact her at Nnyl_ahsatan1
2,1.0,My Hand-Painted Food Rocks,https://i.redd.it/x4unaiiccoq41.jpg,0,You should post this at,These are amazing! So much attention to detail,"Wow, these are incredible! What got you starte...",Oh wow- I might have commented on that complim...,Stoner food? Edit came back to say I love the ...,These are awesome. Also I love that duff and c...,...,Fluff is my fave,I love this so much,Omg! This is really good! Now I want to start ...,0,It truly is super sharp!,This is adorably brilliant!,"This would make a great poster,",that bologna rock lookin like a SNACK,Wow! These are fantastic! And they way this ph...,How cool!
3,2.0,Trying to keep my kids busy! Painters tape + w...,https://i.redd.it/jgnnmda3x9q41.jpg,This is so freaking cool! I'm tempted to do th...,You are a capital G Genius! I don't have littl...,Any kind of markers? Supervisor of a special n...,This is such a good idea for the little ones. ...,The shadows are super pretty but I would be mi...,I love this! And I just looked up “black paint...,I swear my first thought was “I bet that marke...,...,0,This is awesome!,Such a cool idea.,It’s so lovely!,"It doesn't have to be pretty, to be beautiful....",That is quite beautiful,Brilliant!,So cool!,Excuse me while I transform my house.,Absolutely loving the idea! And it's so beauti...
4,3.0,I started this before quarantine and have now ...,https://i.redd.it/socv1a1ttcv41.png,You did an amazing job!,That's so amazing! Everything looks so acurate...,It's an heirloom now!,I was wondering where you got the pattern from...,WHAT THE HECK! this is amazing! i’m so impress...,Amazing dedication to finishing it! Your hands...,Oh I think you missed a thread! LOL.kiding! Th...,...,WOWOWOWOWOW!,This is literally amazing! One of the most bea...,This is insanely beautiful!,Wow!,That's absolutely incredible! What an outstand...,0,That is AMAZING!,WOW amazing. Look at all those colours. what a...,"That is beautiful, I would buy",This is amazing! You are incredibly talented! ...
5,4.0,I made a hoodie! :),https://i.redd.it/ozn6fy0b3tr41.jpg,"Reminder, this is a crafts subredit. You comme...",This is the one you made? It looks good!,"I would wear this in a heartbeat, it looks so ...",This is probably my favorite crocheted piece o...,Love it! Do you have a pattern?,That’s a cutout of a model from magazine right?,That looks so cozy! Wish my crochet skills wer...,...,This looks soo comfy! Definitely going on my t...,You should defo sell that. Looks so profession...,Looks stylish and uber comfy! You did a great ...,"I really like how it fits, great work",Nice,Do you sell anything ? love this,"Great idea, I think I’ll make one!",I thought this was a clothing ad! Amazing. Job...,Cutee! ?,Honestly a dream. Thank you for sharing your p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650,649.0,Wife made this Quilling art for a client on re...,https://i.redd.it/psu7ye81a1a51.jpg,It's stuning,Lovely! Please share on too!,"Your wife is literally SO talented, I loved he...",WOW! So amazing! Does she sell her work anywhe...,This is really beautiful. How much does someth...,Beautiful!,Impressive,...,Thank you so much for the love guys. a sp

In [124]:
result.to_csv(f'C:/Users/Charlie/Desktop/Database/BERT-Sentiment-Analysis-Reddit-API/DataCleaning/filteredData2/{subreddit}.csv', index=False, encoding='utf8')